## NiN实现

&emsp;&emsp;卷积层需要较少的网络参数$c_{i} \times c_{o} \times k^{2}$，但卷积层后的第一个全连接层参数会比较大:


1. `LeNet`: $16 \times 5 \times 5 \times 120 = 48K$。
2. `AlexNet`: $256 \times 5 \times 5 \times 4096 = 26M$。
3. `VGG`: $512 \times 7 \times 7 \times 4096 = 102M$。

&emsp;&emsp;`NiN`尝试不用全连接层。

&emsp;&emsp;一个卷积层后跟两个全连接层，只不过是用$1 \times 1$的卷积核来做的，步幅为`1`, 无填充, 输出形状跟卷积层输出一样, 来起到全连接层的作用。

&emsp;&emsp;交替使用NiN块和步幅为2的最大池化层，逐步减小高宽和增大通道数，最后使用全局平均池化层得到输出，其输入通道数是类别数。

In [1]:
import torch
import torch.nn as nn

def nin_block(in_channels, out_channels, kernel_size, strides, padding):
    """
    in_channels：输入通道大小；
    out_channels：输出通道大小；
    kernel_size：第一个卷积层的核大小；
    strides：第一个卷积层的步幅；
    padding：第一个卷积层的padding
    """
    return nn.Sequential(
        nn.Conv2d(in_channels, out_channels, kernel_size, strides, padding),
        nn.ReLU(),
        nn.Conv2d(out_channels, out_channels, kernel_size=1), nn.ReLU(),
        nn.Conv2d(out_channels, out_channels, kernel_size=1), nn.ReLU())

In [2]:
net = nn.Sequential(
    nin_block(1, 96, kernel_size=11, strides=4, padding=0),
    nn.MaxPool2d(3, stride=2),
    nin_block(96, 256, kernel_size=5, strides=1, padding=2),
    nn.MaxPool2d(3, stride=2),
    nin_block(256, 384, kernel_size=3, strides=1, padding=1),
    nn.MaxPool2d(3, stride=2),
    nn.Dropout(0.5),
    # 标签类别数是10
    nin_block(384, 10, kernel_size=3, strides=1, padding=1),
    nn.AdaptiveAvgPool2d((1, 1)),
    # 将四维的输出转成二维的输出，其形状为(批量大小,10)
    nn.Flatten())

In [3]:
X = torch.randn(size=(1, 1, 224, 224))
for layer in net:
    X = layer(X)
    print(layer.__class__.__name__, "Output shape: \t", X.shape)

Sequential Output shape: 	 torch.Size([1, 96, 54, 54])
MaxPool2d Output shape: 	 torch.Size([1, 96, 26, 26])
Sequential Output shape: 	 torch.Size([1, 256, 26, 26])
MaxPool2d Output shape: 	 torch.Size([1, 256, 12, 12])
Sequential Output shape: 	 torch.Size([1, 384, 12, 12])
MaxPool2d Output shape: 	 torch.Size([1, 384, 5, 5])
Dropout Output shape: 	 torch.Size([1, 384, 5, 5])
Sequential Output shape: 	 torch.Size([1, 10, 5, 5])
AdaptiveAvgPool2d Output shape: 	 torch.Size([1, 10, 1, 1])
Flatten Output shape: 	 torch.Size([1, 10])


## 加载数据集

In [4]:
import torchvision
from torchvision import transforms
from torch.utils import data

def get_dataloader_workers():
    """Use 4 processes to read the data."""
    return 4

def load_data_fashion_mnist(batch_size, resize=None):
    """Download the Fashion-MNIST dataset and then load it into memory."""
    trans = [transforms.ToTensor()]
    
    if resize:
        trans.insert(0, transforms.Resize(resize))
        
    trans = transforms.Compose(trans)
    mnist_train = torchvision.datasets.FashionMNIST(root="../data", train=True, transform=trans, download=True)
    mnist_test = torchvision.datasets.FashionMNIST(root="../data", train=False, transform=trans, download=True)
    
    return (data.DataLoader(mnist_train, batch_size, shuffle=True, num_workers=get_dataloader_workers()),
            data.DataLoader(mnist_test, batch_size, shuffle=False, num_workers=get_dataloader_workers()))

In [5]:
train_iter, test_iter = load_data_fashion_mnist(batch_size=128, resize=224)

/Users/tinyzqh/miniforge3/lib/python3.9/site-packages/torchvision/datasets/mnist.py:498: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  /tmp/pip-req-build-wdh4qigd/torch/csrc/utils/tensor_numpy.cpp:143.)
  return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)


In [6]:
class Accumulator:
    """For accumulating sums over `n` variables."""
    def __init__(self, n):
        self.data = [0.0] * n

    def add(self, *args):
        self.data = [a + float(b) for a, b in zip(self.data, args)]

    def reset(self):
        self.data = [0.0] * len(self.data)

    def __getitem__(self, idx):
        return self.data[idx]

In [7]:
argmax = lambda x, *args, **kwargs: x.argmax(*args, **kwargs)
reduce_sum = lambda x, *args, **kwargs: x.sum(*args, **kwargs)
astype = lambda x, *args, **kwargs: x.type(*args, **kwargs)

def accuracy(y_hat, y):
    """Compute the number of correct predictions."""
    if len(y_hat.shape) > 1 and y_hat.shape[1] > 1:
        y_hat = argmax(y_hat, axis=1)
    cmp = astype(y_hat, y.dtype) == y
    return float(reduce_sum(astype(cmp, y.dtype)))

In [8]:
def evaluate_accuracy_gpu(net, data_iter, device=None):
    """使用GPU计算模型在数据集上的精度"""
    if isinstance(net, torch.nn.Module):
        net.eval()
        if not device: # 如果没有指定device, 就用网络参数中的所给定的那个device。
            device = next(iter(net.parameters())).device
    
    metric = Accumulator(2)
    
    for X, y in data_iter:
        if isinstance(X, list):
            X = [x.to(device) for x in X]  # 将数据挪到device上。
        else:
            X = X.to(device)
            
        y = y.to(device)
        
        metric.add(accuracy(net(X), y), y.numel())
        
        return metric[0] / metric[1]

In [9]:
def train(net, train_iter, test_iter, num_epochs, lr, device):
    def init_weights(m):
        if type(m) == nn.Linear or type(m) == nn.Conv2d:
            nn.init.xavier_uniform_(m.weight)
    
    net.apply(init_weights)  # 对net里面的每一个parameter都去run一下init_weights这个函数。
    print("trainng on {}".format(device))
    
    net.to(device)
    
    optimizer = torch.optim.SGD(net.parameters(), lr=lr)
    loss = nn.CrossEntropyLoss()
    
    num_batches = len(train_iter)
    
    for epoch in range(num_epochs):
        metric = Accumulator(3)
        net.train()
        for i, (X, y) in enumerate(train_iter):
            
            X, y = X.to(device), y.to(device)
            
            y_hat = net(X)
            
            optimizer.zero_grad()
            l = loss(y_hat, y)
            l.backward()
            optimizer.step()
            
            metric.add(l * X.shape[0], accuracy(y_hat, y), X.shape[0])
            
            train_loss = metric[0] / metric[2]
            train_acc = metric[1] / metric[2]
            
            if (i + 1) % (num_batches // 5) == 0 or i == num_batches - 1:
                print("epoch {}, train_loss {}, train_acc {}".format(epoch + (i + 1) / num_batches, 
                                                                  train_loss, train_acc))
    test_acc = evaluate_accuracy_gpu(net, test_iter)
    print(f'loss {train_loss:.3f}, train acc {train_acc:.3f}, 'f'test acc {test_acc:.3f}')

In [10]:
def try_gpu(i=0):
    """Return gpu(i) if exists, otherwise return cpu()."""
    if torch.cuda.device_count() >= i + 1:
        return torch.device(f'cuda:{i}')
    return torch.device('cpu')

In [11]:
lr, num_epochs = 0.1, 10
train(net, train_iter, test_iter, num_epochs, lr, try_gpu())

trainng on cpu
epoch 0.19829424307036247, train_loss 2.3061748396965767, train_acc 0.10131048387096774


Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x1392fb280>
Traceback (most recent call last):
  File "/Users/tinyzqh/miniforge3/lib/python3.9/site-packages/torch/utils/data/dataloader.py", line 1323, in __del__
    def __del__(self):
  File "/Users/tinyzqh/miniforge3/lib/python3.9/site-packages/torch/utils/data/_utils/signal_handling.py", line 66, in handler
    _error_if_any_worker_fails()
RuntimeError: DataLoader worker (pid 31949) is killed by signal: Interrupt: 2. 


KeyboardInterrupt: 